In [ ]:
#https://nlpforhackers.io/wordnet-sentence-similarity/
import re
import csv
import json
import gensim
import threading
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import scipy.interpolate as interpolate
from operator import itemgetter
from scipy import stats
from itertools import groupby
from collections import Counter
from gensim.models import LdaModel
from nltk import pos_tag
from IPython.display import clear_output
from multiprocessing.pool import ThreadPool
from sklearn.feature_extraction.text import TfidfVectorizer
import concurrent.futures
from nltk.corpus import stopwords as nltk_stop_words, wordnet as wn
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from scipy.interpolate import spline
import warnings
warnings.filterwarnings('ignore')
folder_path = 'D:/MS CS/RS DATA/dataset/HSTF/'
tokenizer = RegexpTokenizer(r'\w+')
print("success")

In [ ]:
#Read LDA topics file if needs
lda_title_topics = []
with open(folder_path+'lda_title_topics.json','r', encoding='utf8') as f:
    for doc in f:
        lda_title_topics.append(json.loads(doc))
    f.close()

print(len(lda_title_topics), "Topics successfully readed...")

In [ ]:
lda_paper_topics = []
with open(folder_path+'lda_paper_topics.json','r', encoding='utf8') as f:
    for doc in f:
        lda_paper_topics.append(json.loads(doc))
    f.close()
    
for topic in lda_paper_topics:
    topic['title'] = ' '.join(topic['title'])
    
print(len(lda_paper_topics), "Topics successfully readed...")

### TF-IDF Topic Similarity Corpus

In [ ]:
syn_lda_paper_topics = pd.DataFrame(lda_paper_topics)
syn_lda_paper_topics_groupby_year = syn_lda_paper_topics.groupby("year")
print("success")

In [ ]:
outfile_paper_similarity = open(folder_path+'topic-similarity-corpus.csv', 'w', newline='')
paper_sim_writer = csv.writer(outfile_paper_similarity, quoting=csv.QUOTE_ALL)

def write_paper_similarity(paper_rows):
    paper_sim_writer.writerows(paper_rows)
    outfile_paper_similarity.flush()
    
def get_TFIDF_VS(topic):
    
    similarity_corpus = []
    vect = TfidfVectorizer(min_df=1)
    for year, row in syn_lda_paper_topics_groupby_year:
        
        corpus = row.title.tolist()
        corpus.insert(0, topic['title'])
        tfidf = vect.fit_transform(corpus)
        similarity_corpus.append([topic['id'], year, (sum((tfidf * tfidf.T).A[0])/100)])
        clear_output()
        print(year)

    threading.Thread(target=write_paper_similarity, args=(similarity_corpus,)).start()


if __name__ == "__main__":
    
    print("similarity preprocessing has been started...")
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        for row in lda_topics:
            executor.submit(get_TFIDF_VS, row)
    print("similarity has been successfully measured and saved")

In [ ]:
df_topic_similarity = pd.read_csv(folder_path+'topic-similarity-corpus.csv', names=['tid', 'year', 'sim'])
df_topic_similarity_groupby_year = df_topic_similarity.groupby(['tid'])

norm_corpus = []
for year, row in df_topic_similarity_groupby_year:
    maxScore = row.sim.max()
    minScore = row.sim.min()
    copyMin = minScore.copy()
    for i, rec in row.iterrows():
        rec.sim = (rec.sim - minScore) / (maxScore - minScore)
        if rec.sim == 0:
            rec.sim = copyMin
        norm_corpus.append(rec)
        
pd.DataFrame(norm_corpus).to_csv(folder_path+'norm-topic-similarity-corpus.csv', sep=',', encoding='utf-8', header=None, index=False)
print("success")

#### Check topics is hot or not

In [ ]:
df_topic_similarity = pd.read_csv(folder_path+'norm-topic-similarity-corpus.csv', names=['tid', 'year', 'sim'])
df_topic_similarity_groupby_tid = df_topic_similarity.groupby(['tid'])
print("success")

In [ ]:
# create arrays of fake points
my_xticks = ['2005', '2006', '2007', '2008', '2009']
x = np.array([1, 2, 3,  4,  5])
    
# Polynomial Regression
def polyfit(x, y, degree):
    results = {}
    
    coeffs = np.polyfit(x, y, degree)

     # Polynomial Coefficients
    results['polynomial'] = coeffs.tolist()

    # r-squared
    p = np.poly1d(coeffs)
    
    # fit values, and mean
    yhat = p(x)
    ybar = np.sum(y)/len(y)
    ssreg = np.sum((yhat-ybar)**2)
    sstot = np.sum((y - ybar)**2)
    results['p2_score'] = ssreg / sstot

    return results

#line smoothing
def smoothing(x, deg_val, k_val = 2):
    
    y = np.polyval(deg_val,x)
    x_smooth = np.linspace(x.min(), x.max(), 100)
    tck = interpolate.splrep(x, y, k=k_val)
    y_smooth = interpolate.splev(x_smooth, tck)

    return x_smooth, y_smooth

#for tid, data in grouped_year_similarity:
hot_topic_id = []
cold_topic_id = []
#topic = []
for tid, row_by_topic in df_topic_similarity_groupby_tid:
    
    topic = row_by_topic.sim.tolist()
    
    # fit up to deg=2
    row = polyfit(x, topic, 2)
    r2_score = row['p2_score']
    deg2 = row['polynomial']
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,topic)
    
    #r_value > 0.80
    if p2_score > 0.80:
        
        plt.figure()
        plt.xticks(x, my_xticks)
        x_smooth, y_smooth = smoothing(x, deg2)
        min_val = min(topic)
        if (min_val == topic[2]) and (topic[2]<topic[3]<topic[4]):
            plt.plot(x, topic, marker='s', linestyle=':', color='r', label='Square')
            plt.plot(x_smooth, y_smooth,'r-')
            hot_topic_id.append(tid)
        elif topic[0]<topic[2]<topic[3]<topic[4]:
            plt.plot(x, topic, marker='s', linestyle=':', color='r', label='Square')
            plt.plot(x_smooth, y_smooth,'r-')
            hot_topic_id.append(tid)
        elif (min_val == topic[1] or min_val == topic[2]) and (topic[2]<topic[3]<topic[4]):
            plt.plot(x, topic, marker='s', linestyle=':', color='r', label='Square')
            plt.plot(x_smooth, y_smooth,'r-')
            hot_topic_id.append(tid)
        elif deg2[0] < 0 and slope < 0:
            plt.plot(x, topic, marker='s', linestyle=':', color='b', label='Square')
            plt.plot(x_smooth, y_smooth,'b-')
            cold_topic_id.append(tid)
        elif deg2[0] < 0 and deg2[1] > 0 and r_value > 0.80:
            plt.plot(x, topic, marker='s', linestyle=':', color='b', label='Square')
            plt.plot(x_smooth, y_smooth,'b-')
            cold_topic_id.append(tid)
        elif r_value < 0.80:
            plt.plot(x, topic, marker='s', linestyle=':', color='b', label='Square')
            plt.plot(x_smooth, y_smooth,'b-')
            cold_topic_id.append(tid)
        else:
            plt.plot(x, topic, marker='s', linestyle=':', color='r', label='Square')
            plt.plot(x_smooth, y_smooth,'r-')
            hot_topic_id.append(tid)
            
        plt.xlabel('recognition span')
        plt.ylabel('contribution')
        plt.title(tid)
    else:
        plt.figure()
        plt.xticks(x, my_xticks)
        x_smooth, y_smooth = smoothing(x, deg2)
        
        plt.plot(x, topic, marker='s', linestyle=':', color='b', label='Square')
        plt.plot(x_smooth, y_smooth,'b-')
        cold_topic_id.append(tid)
        
        plt.xlabel('recognition span')
        plt.ylabel('contribution')
        plt.title(tid)        
        
hot_topic_id = pd.DataFrame(hot_topic_id)
hot_topic_id.to_csv(folder_path+'hot_topic_id.csv', sep=',', encoding='utf-8', header=None, index=False)
pd.DataFrame(cold_topic_id).to_csv(folder_path+'cold_topic_id.csv', sep=',', encoding='utf-8', header=None, index=False)
print("Total number of Hot topics are", len(hot_topic_id), ",all another topics can't fulfill criteria")

### TF-IDF Paper Similarity Corpus

In [ ]:
hot_topic_id = pd.read_csv(folder_path+'hot_topic_id.csv', names=['id'])
lda_title_topics = pd.read_json(folder_path+'lda_title_topics.json', orient='records', encoding='utf8', lines=True)
hot_lda_title_topics = lda_title_topics[lda_title_topics.id.isin(hot_topic_id.id.tolist())]

author_corpus = pd.read_json(folder_path+'author_corpus.json', orient='records', encoding='utf8', lines=True)
author_to_paper = pd.read_json(folder_path+'author_to_paper.json', orient='records', encoding='utf8', lines=True)
rs_corpus = pd.read_json(folder_path+'rs_corpus.json', orient='records', encoding='utf8', lines=True)
print("success")

In [ ]:
outfile_paper_similarity = open(folder_path+'author-similarity-corpus2.csv', 'w', newline='')
paper_sim_writer = csv.writer(outfile_paper_similarity, quoting=csv.QUOTE_ALL)

def write_paper_similarity(paper_rows):
    paper_sim_writer.writerows(paper_rows)
    outfile_paper_similarity.flush()

def get_TFIDF_RS(topic):
    
    similarity_corpus = []
    vect = TfidfVectorizer(min_df=1)
    c = 0
    for i, author in author_corpus.iterrows():
        
        rec = rs_corpus[rs_corpus.id.isin(author_to_paper[author_to_paper.aid == author.aid].pid.tolist())]
        year_lst = rec.year.tolist()
        corpus = rec.title.tolist()
        corpus.insert(0, topic['title'])
        tfidf = vect.fit_transform(corpus)
        sim = (tfidf * tfidf.T).A[0]
        for i in range(len(year_lst)):
            print(year_lst[i])
            print(sim[i+1])
            similarity_corpus.append([topic['id'], author['aid'], year_lst[i], sim[i+1]])
        #similarity_corpus.append([topic['id'], author['aid'], (sum(sim)/len(sim))])
        c += 1
        clear_output()
        print(c)

        if c > 4999:
            threading.Thread(target=write_paper_similarity, args=(similarity_corpus,)).start() 
            similarity_corpus.clear()
            c = 0
        
    threading.Thread(target=write_paper_similarity, args=(similarity_corpus,)).start()
        
if __name__ == "__main__":
    
    print("similarity preprocessing has been started...")
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        for i, row in hot_lda_title_topics.iterrows():
            executor.submit(get_TFIDF_RS, row)
    print("similarity has been successfully measured and saved")

### author contribution in papers and rank it by calculating HS Score

In [ ]:
author_corpus_cite = pd.read_json(folder_path+'author_corpus.json', orient='records', encoding='utf8', lines=True)
author_similarity_corpus = pd.read_csv(folder_path+'author-similarity-corpus2.csv', names=['tid', 'aid', 'year', 'score'])
author_similarity_corpus_groupby_aid = author_similarity_corpus.groupby("aid")
print("success")

In [ ]:
wmi_aid = pd.read_csv('D:/MS CS/RS DATA/dataset/WMIRank/aid_wmirank.csv', names=['aid']).aid.tolist()
author_corpus_cite = pd.read_json(folder_path+'author_corpus.json', orient='records', encoding='utf8', lines=True)
author_similarity_corpus = pd.read_csv(folder_path+'author-similarity-corpus2.csv', names=['tid', 'aid', 'year', 'score'])
author_similarity_corpus_groupby_aid = author_similarity_corpus.groupby("aid")

with open(folder_path+'author_corpus.json', 'r', encoding='utf8') as f:
    author_corpus = []
    for doc in f:
        data = json.loads(doc)
        n_citation = int(author_corpus_cite[author_corpus_cite.aid == data['aid']].n_citation)
        n_paper = int(author_corpus_cite[author_corpus_cite.aid == data['aid']].n_paper)
        data['hs_score'] = author_similarity_corpus_groupby_aid.get_group(data['aid']).score.mean()
        del data['p_index']
        del data['pa_index']
        author_corpus.append(data)
    f.close()
author_corpus = pd.DataFrame(author_corpus)
author_corpus.sort_values(by=['hs_score'], inplace=True, ascending=False)
author_corpus.to_json(folder_path+'hs-author-corpus.json', orient='records', lines=True)
print("success")

In [ ]:
author_corpus = pd.read_json(folder_path+'hs-author-corpus.json', orient='records', encoding='utf8', lines=True)
minScore = author_corpus.hs_score.min()
maxScore = author_corpus.hs_score.max()
norm_corpus = []
for i, rec in author_corpus.iterrows():
    rec['n_score'] = (rec.hs_score - minScore) / (maxScore - minScore)
    norm_corpus.append(rec)
    
pd.DataFrame(norm_corpus).to_json(folder_path+'hs-author-corpus.json', orient='records', lines=True)
author_corpus = pd.read_json(folder_path+'hs-author-corpus.json', orient='records', encoding='utf8', lines=True)
author_corpus.sort_values(by=['hs_score'], inplace=True, ascending=False)
print("success")

In [ ]:
author_corpus = pd.read_json('D:/MS CS/RS DATA/dataset/HSRank/hs-author-corpus.json', orient='records', encoding='utf8', lines=True)
author_corpus.sort_values(by=['hs_score'], inplace=True, ascending=False)
author_corpus.head()

### Dummy Testing Code

In [ ]:
wmi_aid = pd.read_csv('D:/MS CS/RS DATA/dataset/WMIRank/aid_wmirank.csv', names=['aid']).aid.tolist()
author_corpus_cite = pd.read_json(folder_path+'author_corpus.json', orient='records', encoding='utf8', lines=True)
author_similarity_corpus = pd.read_csv(folder_path+'author-similarity-corpus2.csv', names=['tid', 'aid', 'year', 'score'])
author_similarity_corpus_groupby_aid = author_similarity_corpus.groupby("aid")

with open(folder_path+'author_corpus.json', 'r', encoding='utf8') as f:
    author_corpus = []
    for doc in f:
        data = json.loads(doc)
        n_citation = int(author_corpus_cite[author_corpus_cite.aid == data['aid']].n_citation)
        n_paper = int(author_corpus_cite[author_corpus_cite.aid == data['aid']].n_paper)
        if data['aid'] in wmi_aid:
            data['hs_score'] = author_similarity_corpus_groupby_aid.get_group(data['aid']).score.mean()*4
        elif (n_citation/n_paper)>15:
            data['hs_score'] = author_similarity_corpus_groupby_aid.get_group(data['aid']).score.mean()
        else:
            data['hs_score'] = author_similarity_corpus_groupby_aid.get_group(data['aid']).score.mean()/2
        del data['p_index']
        del data['pa_index']
        author_corpus.append(data)
    f.close()
author_corpus = pd.DataFrame(author_corpus)
author_corpus.sort_values(by=['hs_score'], inplace=True, ascending=False)
author_corpus.to_json(folder_path+'hs-author-corpus.json', orient='records', lines=True)
print("success")

author_corpus = pd.read_json(folder_path+'hs-author-corpus.json', orient='records', encoding='utf8', lines=True)
minScore = author_corpus.hs_score.min()
maxScore = author_corpus.hs_score.max()
norm_corpus = []
for i, rec in author_corpus.iterrows():
    rec['n_score'] = (rec.hs_score - minScore) / (maxScore - minScore)
    norm_corpus.append(rec)
    
pd.DataFrame(norm_corpus).to_json(folder_path+'hs-author-corpus.json', orient='records', lines=True)
author_corpus = pd.read_json(folder_path+'hs-author-corpus.json', orient='records', encoding='utf8', lines=True)
author_corpus.sort_values(by=['hs_score'], inplace=True, ascending=False)
print("success")